<a href="https://colab.research.google.com/github/ahcamachod/1893-word2vec-interpretacion-del-lenguaje-humano-con-word-embedding/blob/main/Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word2Vec: Interpretación del lenguaje humano con Word Embedding

En este notebook desarrollaremos un clasificador de noticias para una start-up de noticias llamada **Alura Latam News**.

El modelo Word2Vec utilizado para entrenar nuestro clasificador fue tomado de la siguiente fuente: 
 
Aitor Almeida, Aritz Bilbao Jayo. (2018) "Word2vec models for the Spanish Language." Available from: https://github.com/aitoralmeida/spanish_word2vec

## 1. Explorando el Dataset 

## 2. Cargando un modelo Word2Vec

## 3. Entendiendo Word2Vec

## 4. Combinando Vectores

## 5. Clasificando textos